# Basic use of the Emukit



## Overview

In [1]:
# General imports and parameters of figures should be loaded at the beginning of the overview

### General imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

### --- Figure config
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
LEGEND_SIZE = 15
TITLE_SIZE = 25
AXIS_SIZE = 15

The goal of this notebook is to illustrate how the a model can wrapped and used in different tasks in Emukit.

### Navigation

1. [Load your objective, collect some data, build a model](#1.-Load your objective, collect some data, build a model)

2. [Load the elements to solve your problem and run the decision loop if needed](#2.-Load the elements to solve your problem and run the decision loop if needed)

3. [Conclusions](#3.-Conclusions)

## 1.  Load your objective, collect some data, build a model

These steps are common to all methods in Emukit.Here we illustrate how to do it with the branin function.

In [101]:
from emukit.test_functions import branin_function
from emukit.core import ParameterSpace, ContinuousParameter
from emukit.experimental_design.model_free.random_design import RandomDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper

#### Define the objective function

In [87]:
f, _ = branin_function()

#### Define the parameter space

In [88]:
parameter_space = ParameterSpace([ContinuousParameter('x1', -5, 10), ContinuousParameter('x2', 0, 15)])

#### Collect some points and build a model of f

In [89]:
num_data_points = 30
design = RandomDesign(parameter_space)
X = design.get_samples(num_data_points)
Y  = f(X)

#### Fit and wrap a model to the collected data

In [90]:
model_gpy = GPRegression(X,Y)
model_gpy.optimize()
model_emukit = GPyModelWrapper(model_gpy)

## 2.  Load the elements to solve your problem and run the decision loop (if needed)

In [91]:
# -- Decision loops 
from emukit.experimental_design.model_based import ExperimentalDesignLoop
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

# -- Acquisition functions 
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.experimental_design.model_based.acquisitions import ModelVariance

# -- Point calculators
from emukit.core.loop import Sequential

# -- Acquistions optimizers
from emukit.core.optimization import AcquisitionOptimizer

# -- Stopping conditions
from emukit.core.loop import FixedIterationsStoppingCondition

#### Bayesian optimization

In [92]:
## -- Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model = model_emukit)
optimizer            = AcquisitionOptimizer(space = parameter_space)
point_calculator     = Sequential(expected_improvement, optimizer)

In [93]:
## -- Create the Bayesian optimization object
bayesopt_loop = BayesianOptimizationLoop(model = model_emukit,
                                         space = parameter_space,
                                         X_init = X,
                                         Y_init = Y, 
                                         acquisition = expected_improvement,
                                         candidate_point_calculator = point_calculator) 

In [94]:
## -- Run the loop and extract the optimum
stopping_condition = FixedIterationsStoppingCondition(i_max = 10)
bayesopt_loop.run_loop(f, stopping_condition)

#### Experimental design

In [95]:
## -- Load core elements for Experimental design
model_variance   = ModelVariance(model = model_emukit)
optimizer        = AcquisitionOptimizer(space = parameter_space)
point_calculator = Sequential(model_variance, optimizer)

In [96]:
## -- Create the Bayesian optimization object
expdesign_loop = ExperimentalDesignLoop(space = parameter_space,
                                        model = model_emukit,
                                        acquisition = expected_improvement,
                                        update_interval =1,
                                        batch_size = 1) 

In [97]:
## -- Run the loop and extract the optimum
stopping_condition = FixedIterationsStoppingCondition(i_max = 10)
expdesign_loop.run_loop(f, stopping_condition)

#### Bayesian Quadrature

In [100]:
#TODO

#### Snesitivity analysis

In [98]:
### No loop here, compute Sobol indices
senstivity_analysis= MonteCarloSensitivity(model = model_emukit, input_domain = parameter_space)
main_effects, total_effects, _ = senstivity_analysis.compute_effects(num_monte_carlo_points = 10000)

## 3. Conclusions

We have run all these methods using the same model and the sample objective function. With Emukit, probabilistic models can be easily tested over multiple decision problem with a very light API.